### 027 exports vs module.exports
require() 实际上是类似下面的 wrapper 函数。其中第一个参数其实就是 module.exports。
```
fn(module.exports, require, module, __filename, __dirname);
```

In [1]:
(function (exports, require, module, __filename, __dirname) {

    var greet = function() {
        console.log('Hello!');
    };
    
    module.exports = greet;
});

function (exports, require, module, __filename, __dirname) {

    var greet = function() {
        console.log('Hello!');
    };
    
    module.exports = greet;
}

In [2]:
// 模拟 exports 换成其它值
(function() {
 
    var module = {};
    module.exports = {};
    var require = function() {};

    // 模拟 require('./greet.js')
    (function (exports, require, module, __filename, __dirname) {

        // 这里相当于是 greet.js
        exports = function() {
            console.log('Hello!');
        };

        console.log(exports);
        console.log(module.exports);

    })(module.exports, require, module, 'filename', 'dirname');
    return module.exports; // simulate return in require()
})();

function () {
            console.log('Hello!');
        }
{}


{}

上面例子让 exports 指向了不同的对象。**一般可能期望 exports 改了，module.exports 也跟着修改，但这里不是这样。**

In [3]:
// 再模拟，改变 exports 内部（mutate exports），但 exports 本身引用不变
(function() {
 
    var module = {};
    module.exports = {};
    var require = function() {};

    // 模拟 require('./greet.js')
    (function (exports, require, module, __filename, __dirname) {

        // 这里相当于是 greet.js
        // NOTE: 由于某些原因，函数并不会输出，所以又加了字符串属性来验证
        exports.greet = function() { console.log('Hello!'); };
        exports.greetStr = `function() { console.log('Hello!'); }`;

        console.log(exports);
        console.log(module.exports);

    })(module.exports, require, module, 'filename', 'dirname');
    return module.exports; // simulate return in require()
})();

{"greetStr":"function() { console.log('Hello!'); }"}
{"greetStr":"function() { console.log('Hello!'); }"}


{"greetStr":"function() { console.log('Hello!'); }"}

这里容易造成误解，所以简单的方法：***Just use module.exports**

### 028 Requiring Native Core Modules

In [4]:
var util = require('util');

var name = 'Tony';
var greeting = util.format('Hello, %s', name);
//util.log(greeting); // jupyter 里面似乎不工作
console.log(greeting);

Hello, Tony


如果自己也有同名的 module 怎么办？问题不大，自己的使用相对路径：
```
require('./util');
```
但是还是不建议使用相同的名字，造成混淆。